In [42]:
import pandas as pd
import numpy as np
import chromedriver_autoinstaller
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [43]:
url = f"https://www.idx.co.id/en/listed-companies/company-profiles/"
            
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
try: # 크롬 드라이버
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
        # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')


In [44]:
driver.get(url)

In [45]:
driver.maximize_window()

In [47]:
select_class = Select(driver.find_element(By.TAG_NAME, 'select'))

In [48]:
select_class.select_by_value("-1")

In [49]:
table_element = driver.find_element(By.XPATH, '//*[@id="vgt-table"]')

In [50]:
table_html = table_element.get_attribute('outerHTML')

In [51]:
df = pd.read_html(table_html)[0]

In [52]:
df = df.drop("Unnamed: 0", axis=1)

In [53]:
df.columns

Index(['Code or Company's Name Sort table by Code or Company's Name in descending order',
       'Name Sort table by Name in descending order',
       'Listing Date Sort table by Listing Date in descending order'],
      dtype='object')

In [54]:
company_name_lst = list(df["Code or Company's Name Sort table by Code or Company's Name in descending order"])

In [55]:
for num, name in tqdm(enumerate(company_name_lst)):
    try:
        driver.get(url+name)
        driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[2]/div[1]/nav/button[6]').click()
        select_list = Select(driver.find_element(By.TAG_NAME, 'select'))
        select_list.select_by_value("-1")
        table = driver.find_element(By.XPATH, '//*[@id="vgt-table"]')
        table_html_idx = table.get_attribute('outerHTML')
        if num == 0: company_df = pd.read_html(table_html_idx)[0]
        else: company_df = company_df.append(pd.read_html(table_html_idx)[0])
        time.sleep(2)
    except: pass

1it [00:02,  2.43s/it]C:\Users\user\AppData\Local\Temp\ipykernel_37696\2982253751.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: company_df = company_df.append(pd.read_html(table_html_idx)[0])
2it [00:04,  2.41s/it]C:\Users\user\AppData\Local\Temp\ipykernel_37696\2982253751.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: company_df = company_df.append(pd.read_html(table_html_idx)[0])
3it [00:07,  2.72s/it]C:\Users\user\AppData\Local\Temp\ipykernel_37696\2982253751.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: company_df = company_df.append(pd.read_html(table_html_idx)[0])
4it [00:10,  2.67s/it]C:\Users\user\AppData\Local\Temp\ipykernel_37696\2982253751.py:10: FutureWarning: The frame.appe

In [ ]:
company_df.to_excel("test.xlsx", index=False)